In [ ]:
import yfinance as yf   
import pandas as pd
import matplotlib.pyplot as plt
import mplcyberpunk

from regras_de_negocio import *

plt.style.use("cyberpunk")


In [ ]:
ticker = "BTC-USD"
dados = yf.download(ticker, auto_adjust=False)
dados = dados.dropna()
dados = dados.droplevel(1, axis=1)
dados

Preparar e calcular dados importantes para o modelo

In [ ]:
dados["media_maxima"] = dados.High.rolling(window=20).mean(20)
dados["media_minima"] = dados.Low.rolling(window=20).mean(20)
dados.head(30)

Gerar sinais de compra

In [ ]:
dados["sinal_compra"] = 0
dados.sinal_compra = (dados.Close > dados.media_maxima).astype(int)
dados

Gerar sinais de venda

In [ ]:
dados["sinal_venda"] = 0
dados.sinal_venda = (dados.Close < dados.media_minima).astype(int)
dados

Criar colunas necessárias para operacoes

In [ ]:
dados["posicao"] = False
dados["trade"] = 0
dados["posicao_obs"] = ""
dados["retorno_modelo"] = 0.0

Gerar Operacoes

In [ ]:
dados = gerar_posicao_e_trade_obs(dados, 'sinal_compra', 'sinal_venda', 'posicao', 'trade', 'posicao_obs')


Calcular retorno de todos os trades e retorno da base de comparacao

In [ ]:
# retorno do modelo
dados = calcular_retorno_modelo(dados, 'Close', 'posicao_obs', 'retorno_modelo')

# retorno base de comparacao
dados["retorno"] = dados.Close.pct_change()


Gerar variaveis para plotagem dos dados

In [ ]:
dados_retorno_modelo = (1 + dados.retorno_modelo).cumprod()-1
dados_retorno = (1 + dados.retorno).cumprod()-1

In [ ]:
dados_retorno_modelo.plot(label = "Modelo")
dados_retorno.plot(label = "Base de Comparação")
plt.legend()
plt.grid()
plt.show()